In [1]:
import sys
sys.path.append('../../')
import kitorch as mt
from kitorch import nn,optim
from kitorch import functional as F
import numpy as np
import torch

In [2]:
class LSTM(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,dropout=0):
        super(LSTM, self).__init__()

        self.lstm = nn.LSTM(
            input_size= input_size,
            hidden_size= hidden_size,
            num_layers= num_layers,
            dropout=dropout,
        )
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self,x,h0,c0):
        x, hn,cn = self.lstm(x,h0,c0)
        x = self.fc(x)
        return x,hn,cn
    

class TrochLSTM(torch.nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,dropout=0):
        super(TrochLSTM, self).__init__()

        self.lstm = torch.nn.LSTM(
            input_size= input_size,
            hidden_size= hidden_size,
            num_layers= num_layers,
            dropout = dropout
      )
        self.fc = torch.nn.Linear(hidden_size, 1)
        
    def forward(self,x,h0,c0):
        x, (hn,cn) = self.lstm(x,(h0,c0))
        x = self.fc(x)
        return x,hn ,cn  
    


In [3]:
import matplotlib.pyplot as plt
# 创造一些数据
steps = np.linspace(0, np.pi*2, 100, dtype=np.float)
x_np = np.sin(steps)
y_np = np.cos(steps)
#
# “看”数据
plt.plot(steps, y_np, 'r-', label='target(cos)')
plt.plot(steps, x_np, 'b-', label='input(sin)')
plt.legend(loc='best')
plt.show()

<Figure size 640x480 with 1 Axes>

In [4]:
def normalize_weights(parameters):
    for layer_paras in parameters:
        for para in layer_paras:
            norm = np.linalg.norm(para.grad.data)
            if norm > 1000:
                para.grad.data/=norm

In [5]:
# 定义优化器和损失函数
lr = 0.001
num_layers=2
dropout=0.1
model = LSTM(1,32,num_layers,dropout=dropout)
torch_model = TrochLSTM(1,32,num_layers,dropout=dropout)
optimizer = optim.RMSprop(model.parameters(), lr=lr)
torch_optimizer = torch.optim.RMSprop(torch_model.parameters(), lr=lr)

In [6]:
%matplotlib qt5

In [7]:
%%time
seq_len = 20
input_size = 1
h_state = mt.zeros(num_layers,1,32)
c_state = mt.zeros(num_layers,1,32)
torch_h_state = torch.zeros(num_layers,1,32)
torch_c_state = torch.zeros(num_layers,1,32)
# plt.figure(1,figsize=(12,5))
# 图出来了，可以继续画


for epoch in range(300):
# epoch = 0

    start, end = epoch * np.pi, (epoch+1)*np.pi

    steps = np.linspace(start, end, seq_len, dtype=np.float32)
    x_np = np.sin(steps)
    y_np = np.cos(steps)

    x = mt.from_numpy(x_np[:,np.newaxis,np.newaxis])
    y = mt.from_numpy(y_np[:,np.newaxis,np.newaxis])

    optimizer.zero_grad()  
    prediction, h_state,c_state = model(x, h_state,c_state)
    h_state,c_state = h_state.copy(),c_state.copy()
    loss = F.mse_loss(prediction, y)

    loss.backward()
#     normalize_weights(model.parameters())
    optimizer.step()
    

    # torch RNN
    x = torch.from_numpy(x_np[:,np.newaxis,np.newaxis])
    y = torch.from_numpy(y_np[:,np.newaxis,np.newaxis])
    torch_optimizer.zero_grad()  
    torch_prediction, torch_h_state, torch_c_state = torch_model(x, torch_h_state,torch_c_state)
    torch_h_state = torch_h_state.data
    torch_c_state = torch_c_state.data
    torch_loss = torch.nn.functional.mse_loss(torch_prediction, y)
    torch_loss.backward()
    torch_optimizer.step()
    
    if epoch%50 == 0:
        print(loss.data.item(),torch_loss.data.item())
        
        
    plt.plot(steps,y_np.flatten(),'r-')  
    plt.plot(steps,prediction.data.flatten(),'b-')
    plt.plot(steps,torch_prediction.data.numpy().flatten(),'k-')
    plt.pause(0.001)
    if epoch%50 == 0:
        plt.cla()
    
    
plt.show()  

0.563545512916664 0.5347996950149536
0.05224697262459098 0.009309515357017517
0.01538404436844209 0.0017448797589167953
0.002710258376745477 0.0007190348696894944
0.0014605663502091158 0.0012761395191773772
0.001322508118811801 0.0013036486925557256
CPU times: user 43.2 s, sys: 12.8 s, total: 56 s
Wall time: 13.4 s
